In [ ]:
%matplotlib qt
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (12, 5)
# plt.rcParams.update({"font.size": 14})
import numpy as np
from xbout import open_boutdataset

In [ ]:
ds = open_boutdataset(chunks={"t": 4},datapath="/Users/power8/Documents/01_code/04_bout/churning_mode/data/BOUT.dmp.*.nc")

# Use squeeze() to get rid of the y-dimension, which has length 1 as blob2d does not
# simulate the parallel dimension.
ds = ds.squeeze(drop=True)
dx = ds["dx"].isel(x=0).values

x = "x"
y = "y"

# Get rid of existing "x" coordinate, which is just the index values.
ds = ds.drop(x)
ds = ds.drop(y)

# Create a new coordinate, which is length in units of rho_s
ds = ds.assign_coords(x=np.arange(ds.sizes[x])*dx)
ds = ds.assign_coords(y=np.arange(ds.sizes[y])*dx)

In [ ]:
##### ANIMATE LIST OF VARIABLES
ds.bout.animate_list(["omega", "P", "psi", "phi", "u_x", "u_y"])
# ds.bout.animate_list(["P", "omega"], vmin=0.5, vmax=1.5,aspect='equal')
# ds.bout.animate_list(["omega"], vmin=0,vmax=0.1)

In [ ]:
##### PLOT COLOURMAPS OF SEVERAL VARIABLES AT GIVEN TIMESTAMP #####
t = -1
vars = ["P", "psi", "omega", "phi"]
fig,ax = plt.subplots(ncols=len(vars))
for i,v in enumerate(vars):
    if v == "P" or v == "psi":
        vmin = ds[v][0].values.min()
        vmax = ds[v][0].values.max()
    else:
        vmin = None 
        vmax = None
    ax[i].contourf(ds[x],ds[y],ds[v][t].values.T, vmin=vmin, vmax = vmax,levels=20)
    ax[i].set_xlabel(x)
    ax[i].set_ylabel(y)
    ax[i].set_title(v)
fig.tight_layout()

In [ ]:
##### ANIMATE MIXED COLOURMAPS AND LINE PLOTS #####
ds.bout.animate_list(["P","psi",ds["phi"].isel(x=50)])

In [ ]:
##### ANIMATE CONTOUR PLOTS OF PSI #####
from matplotlib import animation
# Generate grid for plotting
xmin = 52
xmax = 80
ymin = xmin-2
ymax = xmax-2
xvals = ds[x][xmin:xmax]
yvals = ds[y][xmin:xmax]
v = "psi"
var = ds[v].values[10:50,xmin:xmax,xmin:xmax]

fig = plt.figure()
ax = plt.axes(xlim=(0, xvals.values.max()), ylim=(0, yvals.values.max()),aspect='equal')  
vmin = var[0].min()
vmax = var[0].max()
levels = np.sort(list(np.linspace(vmin,vmax,20)) + [0])

# animation function
def animate(i): 
    ax.clear()
    z = var[i].T
    cont = ax.contourf(xvals, yvals, z, vmin=vmin, vmax=vmax, levels=levels)
    if (i == 0):
        ax.set_title(i)
    else:
        ax.set_title(i)
    ax.set_xlabel(x)
    ax.set_ylabel(y)

    return cont  

anim = animation.FuncAnimation(fig, animate, frames=var.shape[0])
anim.save("churn.gif")


In [ ]:
##### PLOT CONTOUR OF PSI AT FIRST AND LAST TIMESTAMP #####
fig,ax = plt.subplots(1)
levels = 20
v = "psi"
t1 = 0
t2 = 3
ax.contour(ds[x],ds[y],ds[v][t1].values.T, levels=levels)
ax.contour(ds[x],ds[y],ds[v][t2].values.T, linestyles="--", levels=levels)